In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x) 

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def g(x):  
    x_temp = torch.cos(pi*x)
    g_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return g_x

In [10]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x).cuda()
    uxx = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda()
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) + model(xa1) - g(xa1))**2).cuda()
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) + model(xa2) - g(xa2))**2).cuda()
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) + model(xb1) - g(xb1))**2).cuda()
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) + model(xb2) - g(xb2))**2).cuda()
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) + model(xc1) - g(xc1))**2).cuda()
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) + model(xc2) - g(xc2))**2).cuda()
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) + model(xd1) - g(xd1))**2).cuda()
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) + model(xd2) - g(xd2))**2).cuda()
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 500
    return part_1 + lambda1 * part_2 / 8

In [11]:
Data_size = 4000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [12]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [13]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [14]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
import torch.optim as optim
import torch.nn as nn
import time

In [16]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [17]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [18]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Robin_batch_size_4000.npy", error_save)
np.save("DGM_relative_error_4D_Robin_batch_size_4000.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(2049.1447, device='cuda:0')
current relative error is:  tensor(0.9722, device='cuda:0')
current epoch is:  50
current loss is:  tensor(1848.7573, device='cuda:0')
current relative error is:  tensor(0.8498, device='cuda:0')
current epoch is:  100
current loss is:  tensor(1311.9664, device='cuda:0')
current relative error is:  tensor(0.5085, device='cuda:0')
current epoch is:  150
current loss is:  tensor(733.2807, device='cuda:0')
current relative error is:  tensor(0.5910, device='cuda:0')
current epoch is:  200
current loss is:  tensor(483.4118, device='cuda:0')
current relative error is:  tensor(0.6800, device='cuda:0')
current epoch is:  250
current loss is:  tensor(328.3042, device='cuda:0')
current relative error is:  tensor(0.6744, device='cuda:0')
current epoch is:  300
current loss is:  tensor(224.5003, device='cuda:0')
current relative error is:  tensor(0.6446, device='cuda:0')
current epoch is:  350
current loss is:  tensor(164.281

current epoch is:  3100
current loss is:  tensor(2.3833, device='cuda:0')
current relative error is:  tensor(0.2025, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(2.3466, device='cuda:0')
current relative error is:  tensor(0.2047, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(2.0811, device='cuda:0')
current relative error is:  tensor(0.2103, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(2.0336, device='cuda:0')
current relative error is:  tensor(0.2038, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(1.9334, device='cuda:0')
current relative error is:  tensor(0.2013, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(1.8925, device='cuda:0')
current relative error is:  tensor(0.2167, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(2.0089, device='cuda:0')
current relative error is:  tensor(0.2012, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(1.8878, devic

current epoch is:  6200
current loss is:  tensor(0.9869, device='cuda:0')
current relative error is:  tensor(0.2121, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(0.8993, device='cuda:0')
current relative error is:  tensor(0.1998, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(0.7773, device='cuda:0')
current relative error is:  tensor(0.2034, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(0.6631, device='cuda:0')
current relative error is:  tensor(0.2059, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(0.7801, device='cuda:0')
current relative error is:  tensor(0.2124, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(1.3267, device='cuda:0')
current relative error is:  tensor(0.2138, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(0.7685, device='cuda:0')
current relative error is:  tensor(0.1885, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(0.8316, devic

current epoch is:  9300
current loss is:  tensor(0.7401, device='cuda:0')
current relative error is:  tensor(0.2013, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.4252, device='cuda:0')
current relative error is:  tensor(0.1925, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.4203, device='cuda:0')
current relative error is:  tensor(0.1908, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.4128, device='cuda:0')
current relative error is:  tensor(0.1928, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.4965, device='cuda:0')
current relative error is:  tensor(0.1902, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.4314, device='cuda:0')
current relative error is:  tensor(0.1838, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.4498, device='cuda:0')
current relative error is:  tensor(0.1907, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.8601, devic

current epoch is:  12350
current loss is:  tensor(0.4411, device='cuda:0')
current relative error is:  tensor(0.1795, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.6929, device='cuda:0')
current relative error is:  tensor(0.1675, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.6506, device='cuda:0')
current relative error is:  tensor(0.1865, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.3668, device='cuda:0')
current relative error is:  tensor(0.1768, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.5042, device='cuda:0')
current relative error is:  tensor(0.1865, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.4993, device='cuda:0')
current relative error is:  tensor(0.1655, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.4413, device='cuda:0')
current relative error is:  tensor(0.1688, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.313

current epoch is:  15400
current loss is:  tensor(0.2994, device='cuda:0')
current relative error is:  tensor(0.1630, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.2861, device='cuda:0')
current relative error is:  tensor(0.1647, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.3184, device='cuda:0')
current relative error is:  tensor(0.1540, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.4904, device='cuda:0')
current relative error is:  tensor(0.1546, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.4173, device='cuda:0')
current relative error is:  tensor(0.1595, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.2782, device='cuda:0')
current relative error is:  tensor(0.1676, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.3949, device='cuda:0')
current relative error is:  tensor(0.1713, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.257

current epoch is:  18450
current loss is:  tensor(0.5869, device='cuda:0')
current relative error is:  tensor(0.1431, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.3440, device='cuda:0')
current relative error is:  tensor(0.1479, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.4261, device='cuda:0')
current relative error is:  tensor(0.1691, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.2469, device='cuda:0')
current relative error is:  tensor(0.1568, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.5131, device='cuda:0')
current relative error is:  tensor(0.1436, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.2998, device='cuda:0')
current relative error is:  tensor(0.1500, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(1.5039, device='cuda:0')
current relative error is:  tensor(0.1779, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.299

In [19]:
error_save_append = np.zeros(traintime)
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save_append[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Robin_batch_size_4000_append.npy", error_save_append)
np.save("DGM_relative_error_4D_Robin_batch_size_4000_append.npy", error_save_append)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(0.2199, device='cuda:0')
current relative error is:  tensor(0.1584, device='cuda:0')
current epoch is:  50
current loss is:  tensor(0.5530, device='cuda:0')
current relative error is:  tensor(0.1644, device='cuda:0')
current epoch is:  100
current loss is:  tensor(0.5130, device='cuda:0')
current relative error is:  tensor(0.1422, device='cuda:0')
current epoch is:  150
current loss is:  tensor(0.3521, device='cuda:0')
current relative error is:  tensor(0.1562, device='cuda:0')
current epoch is:  200
current loss is:  tensor(0.2299, device='cuda:0')
current relative error is:  tensor(0.1470, device='cuda:0')
current epoch is:  250
current loss is:  tensor(0.4549, device='cuda:0')
current relative error is:  tensor(0.1691, device='cuda:0')
current epoch is:  300
current loss is:  tensor(0.2578, device='cuda:0')
current relative error is:  tensor(0.1441, device='cuda:0')
current epoch is:  350
current loss is:  tensor(0.2118, device='cuda:0')

current epoch is:  3100
current loss is:  tensor(0.3014, device='cuda:0')
current relative error is:  tensor(0.1447, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(0.3443, device='cuda:0')
current relative error is:  tensor(0.1371, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(0.2235, device='cuda:0')
current relative error is:  tensor(0.1469, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(0.4697, device='cuda:0')
current relative error is:  tensor(0.1534, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(0.4589, device='cuda:0')
current relative error is:  tensor(0.1287, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(0.2134, device='cuda:0')
current relative error is:  tensor(0.1466, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(0.2291, device='cuda:0')
current relative error is:  tensor(0.1465, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(0.2055, devic

current epoch is:  6200
current loss is:  tensor(0.1864, device='cuda:0')
current relative error is:  tensor(0.1370, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(0.2938, device='cuda:0')
current relative error is:  tensor(0.1261, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(0.7881, device='cuda:0')
current relative error is:  tensor(0.1531, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(0.2568, device='cuda:0')
current relative error is:  tensor(0.1403, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(0.2202, device='cuda:0')
current relative error is:  tensor(0.1364, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(0.1856, device='cuda:0')
current relative error is:  tensor(0.1328, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(0.2062, device='cuda:0')
current relative error is:  tensor(0.1315, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(0.2045, devic

current epoch is:  9300
current loss is:  tensor(0.1549, device='cuda:0')
current relative error is:  tensor(0.1314, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.1612, device='cuda:0')
current relative error is:  tensor(0.1237, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.5390, device='cuda:0')
current relative error is:  tensor(0.1401, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.1850, device='cuda:0')
current relative error is:  tensor(0.1225, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.2136, device='cuda:0')
current relative error is:  tensor(0.1208, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.3229, device='cuda:0')
current relative error is:  tensor(0.1215, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.1754, device='cuda:0')
current relative error is:  tensor(0.1257, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.3098, devic

current epoch is:  12350
current loss is:  tensor(0.1685, device='cuda:0')
current relative error is:  tensor(0.1164, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.1571, device='cuda:0')
current relative error is:  tensor(0.1239, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.1498, device='cuda:0')
current relative error is:  tensor(0.1249, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.1639, device='cuda:0')
current relative error is:  tensor(0.1202, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.1572, device='cuda:0')
current relative error is:  tensor(0.1219, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.1937, device='cuda:0')
current relative error is:  tensor(0.1303, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.2213, device='cuda:0')
current relative error is:  tensor(0.1256, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.153

current epoch is:  15400
current loss is:  tensor(0.2716, device='cuda:0')
current relative error is:  tensor(0.1245, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.1648, device='cuda:0')
current relative error is:  tensor(0.1167, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.4017, device='cuda:0')
current relative error is:  tensor(0.1052, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.3162, device='cuda:0')
current relative error is:  tensor(0.1073, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.5190, device='cuda:0')
current relative error is:  tensor(0.1262, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.2603, device='cuda:0')
current relative error is:  tensor(0.1096, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.1502, device='cuda:0')
current relative error is:  tensor(0.1165, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.534

current epoch is:  18450
current loss is:  tensor(0.1237, device='cuda:0')
current relative error is:  tensor(0.1081, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.1448, device='cuda:0')
current relative error is:  tensor(0.1087, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.1361, device='cuda:0')
current relative error is:  tensor(0.1119, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.1240, device='cuda:0')
current relative error is:  tensor(0.1092, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.1836, device='cuda:0')
current relative error is:  tensor(0.1131, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.3826, device='cuda:0')
current relative error is:  tensor(0.0973, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.2130, device='cuda:0')
current relative error is:  tensor(0.1131, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.184